In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
import pickle

In [2]:
#load the dataset
import pandas as pd
data= pd.read_csv("Churn_Modelling.csv")
data.head()

# Preprocessing data
# load the dataset
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
# # Preprocessing data
# # load the dataset
# data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
# data


In [4]:
#encode the categorical data
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
#Gender Encoding
le_encoder_gender=LabelEncoder()
data["Gender"]=le_encoder_gender.fit_transform(data["Gender"])
le_encoder_gender

#Geography encoding
onehot_encoder_geo=OneHotEncoder()
geo_encoded=onehot_encoder_geo.fit_transform([["Geography"]])


In [5]:
#Creating dataframes with proper column names
geo_columns=onehot_encoder_geo.get_feature_names_out(["Geography"])
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=geo_columns)

In [6]:
#Combine onehot encoded with original data
data=pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Geography
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,NaN
2,502,0,42,8,159660.80,3,1,0,113931.57,1,NaN
3,699,0,39,1,0.00,2,0,0,93826.63,0,NaN
4,850,0,43,2,125510.82,1,1,1,79084.10,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,NaN
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,NaN
9997,709,0,36,7,0.00,1,0,1,42085.58,1,NaN
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,NaN


In [7]:
#save the pickle files
with open ("le_encoder_gender.pkl","wb") as file:
    pickle.dump(le_encoder_gender,file)

with open ("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file)

In [8]:
# Divide into independent (X) and dependent (y) features
X = data.drop("Exited", axis=1)
y = data["Exited"]

# Split into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

# Scaling the features
from sklearn.preprocessing import StandardScaler
import pickle

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

# Save the scaler
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)


c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [10]:
#Building the neural networks
model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train_scaled.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1,activation="sigmoid")
])
#compile with optimizers
import tensorflow as tf
optimizers=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(
    optimizer=optimizers,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
#setup tensorboard call back
import datetime
log_dir="log/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [12]:
#setup Early stopping
early_stopping_callback=EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [13]:
#ensuring the model is compiled
model.compile(
    optimizer=optimizers,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

#train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7945 - loss: 0.5479 - val_accuracy: 0.8035 - val_loss: 0.4976
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7945 - loss: 0.5083 - val_accuracy: 0.8035 - val_loss: 0.4957
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7945 - loss: 0.5081 - val_accuracy: 0.8035 - val_loss: 0.4961
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7945 - loss: 0.5081 - val_accuracy: 0.8035 - val_loss: 0.4957
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7945 - loss: 0.5081 - val_accuracy: 0.8035 - val_loss: 0.4959
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7945 - loss: 0.5082 - val_accuracy: 0.8035 - val_loss: 0.4959
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7945 - loss: 0.5080 - val_accuracy: 0.8035 - val_loss: 0.4957
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7945 - loss: 0.5081 - val_accu

In [14]:
#saving the model
model.save("model.h5")

In [15]:
# launch the tensorboard extension
%load_ext tensorboard

In [16]:
# %tensorboard--logdir log/fit20250812-004020/train/